# Grand canonical ensemble transition-matrix Monte Carlo with SPC/E

This example is similar to the Lennard Jones tutorial, except this time we simulate the SPC/E water model.

In [1]:
import sys
import unittest
import feasst
sys.path.insert(0, feasst.install_dir() + '/plugin/flat_histogram/tutorial/')
import fh
sys.path.insert(0, feasst.install_dir() + '/plugin/ewald/tutorial/')
import spce

class TestFlatHistogramSPCE(unittest.TestCase):
    """Test flat histogram grand canonical ensemble Monte Carlo simulations"""
    def test_serial_5max(self):
        """Compare the free energies and potential energies with the NIST SRSW
        https://www.nist.gov/programs-projects/nist-standard-reference-simulation-website
        https://mmlapps.nist.gov/srs/LJ_PURE/eostmmc.htm
        """
        config = feasst.Configuration(
            feasst.MakeDomain(feasst.args({"cubic_box_length": "20"})),
            feasst.args({"particle_type": feasst.install_dir() +  "/forcefield/data.spce",
                         "physical_constants": "CODATA2010"}))
        temperature = feasst.kelvin2kJpermol(525, config)
        criteria = fh.criteria_flathist(temperature=temperature,
                                        chemical_potential=-8.14*temperature,
                                        macro_min=0, macro_max=5, tmmc=True)
        monte_carlo = fh.monte_carlo(criteria=criteria,
                                     system=spce.system(config))
        
        lnpi_previous = [
            [-2.7207, 0.015],
            [-1.8523, 0.015],
            [-1.54708, 0.016],
            [-1.51786, 0.015],
            [-1.6479, 0.015],
            [-1.8786, 0.03]]
        energy_previous = [
            [0, 1e-13],
            [-0.0879115, 1.1293158298007674394e-05],
            [-2.326, 0.12],
            [-6.806, 0.24],
            [-13.499, 0.5],
            [-22.27, 1.0]]
        for macro in range(criteria.num_states()):
            self.assertAlmostEqual(
                lnpi_previous[macro][0],
                criteria.bias().ln_prob().value(macro),
                delta=15.*lnpi_previous[macro][1]
            )
            energy_analyzer = monte_carlo.analyze(monte_carlo.num_analyzers() - 1)
            energy_accumulator = energy_analyzer.analyze(macro).accumulator()
            stdev = (energy_previous[macro][1]**2 + energy_accumulator.block_stdev()**2)**(1./2.)
            self.assertAlmostEqual(
                energy_previous[macro][0],
                energy_accumulator.average(),
                #criteria.bias().ln_prob().value(macro),
                delta=15*stdev
            ) 


In [2]:
%%time
unittest.main(argv=[''], verbosity=2, exit=False)

test_serial_5max (__main__.TestFlatHistogramSPCE)
Compare the free energies and potential energies with the NIST SRSW ... ok

----------------------------------------------------------------------
Ran 1 test in 104.559s

OK


CPU times: user 1min 43s, sys: 1.08 s, total: 1min 44s
Wall time: 1min 44s


Did this tutorial work as expected? Did you find any inconsistencies or have any comments? Please [contact](../../../CONTACT.rst) us. Any feedback is appreciated!